<a href="https://colab.research.google.com/github/totti0223/deep_learning_for_biologists_with_keras/blob/master/notebooks/basic_usage_of_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Basic Codes to train and run CNN model in Keras
- for clarity, the library import commands will be redundant throughout the notebook

## Notebook and data preparation 

In [1]:
#we will use the rice seed image datset as an example
!apt-get install subversion > /dev/null
!svn export https://github.com/totti0223/deep_learning_for_biologists_with_keras/trunk/notebooks/data/image image > /dev/null
    
#to make sure keras imagedatagenerator flow from dataframe works
!pip uninstall keras-preprocessing -y > /dev/null
!pip install git+https://github.com/keras-team/keras-preprocessing.git > /dev/null

svn: E155000: Destination directory exists; please remove the directory or use --force to overwrite
svn: E155000: 'image' already exists


In [2]:
#check the directory structure
import os

for dirpath, dirnames, filenames, in os.walk("image"):
    print(dirpath)
    if dirnames:
        print(dirnames)
    if filenames:
        print(filenames)
    print("__________")

image
['train', 'test']
['train.txt', 'test.txt']
__________
image/train
['proper', 'broken']
__________
image/train/proper
['37.jpg', '66.jpg', '87.jpg', '117.jpg', '99.jpg', '31.jpg', '60.jpg', '116.jpg', '57.jpg', '77.jpg', '134.jpg', '73.jpg', '200.jpg', '183.jpg', '17.jpg', '80.jpg', '55.jpg', '72.jpg', '24.jpg', '38.jpg', '111.jpg', '41.jpg', '192.jpg', '151.jpg', '29.jpg', '32.jpg', '113.jpg', '50.jpg', '132.jpg', '94.jpg', '140.jpg', '154.jpg', '54.jpg', '108.jpg', '95.jpg', '153.jpg', '20.jpg', '21.jpg', '42.jpg', '51.jpg', '89.jpg', '84.jpg', '30.jpg', '104.jpg', '198.jpg', '19.jpg', '112.jpg', '12.jpg', '115.jpg', '107.jpg', '27.jpg', '23.jpg', '131.jpg', '90.jpg', '138.jpg', '85.jpg', '34.jpg', '102.jpg', '67.jpg', '76.jpg', '18.jpg', '82.jpg', '40.jpg', '35.jpg', '74.jpg', '49.jpg', '197.jpg', '22.jpg', '58.jpg', '14.jpg', '79.jpg', '127.jpg', '96.jpg', '39.jpg', '106.jpg', '136.jpg', '25.jpg', '93.jpg', '129.jpg', '28.jpg', '98.jpg', '16.jpg', '56.jpg', '118.jpg', '190.jp

# 1. Loading the Dataset

- For now, we will load an image to an single array variable with 

    (batch,32,32,3) : (number of images, width px, height px, rgb channel dimentions)

    in case of gray scale images, it'll be : (batch,32,32,1)
    
- For class label, we define an array of shape 2: [1, 0], which the first element is the probability of the image being "proper" class, while the second as "broken class".
    So the label paired with each image will be either assigned an array of [1,0] or [0,1]

## 1-1. Everything to memory

In [3]:
import os
import numpy as np

from keras.preprocessing.image import load_img, img_to_array
from keras.utils.np_utils import to_categorical

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


### 1-1-1. Relatively manual

In [0]:
#note excuse for the .DS_Store contamination in the dataset. to remove the file after obtaining lists using os.listdir,
#do..... paths = [x for x in paths if x.endswith(".jpg")]

x = []
y = []

test_x = []
test_y = []

paths = os.listdir("image/train/proper")
paths = [x for x in paths if x.endswith(".jpg")]
for path in paths:
    img_path = os.path.join("image/train/proper",path)
    img = load_img(img_path, grayscale=False, color_mode='rgb', target_size=(32,32), interpolation='nearest')
    img = img_to_array(img)
    img /= 255 
    x.append(img)
    y.append([1,0])
    
paths = os.listdir("image/train/broken")
paths = [x for x in paths if x.endswith(".jpg")]
for path in paths:
    img_path = os.path.join("image/train/broken",path)
    img = load_img(img_path, grayscale=False, color_mode='rgb', target_size=(32,32), interpolation='nearest')
    img = img_to_array(img)
    img /= 255 
    x.append(img)
    y.append([0,1])

paths = os.listdir("image/test/proper")
paths = [x for x in paths if x.endswith(".jpg")]
for path in paths:
    img_path = os.path.join("image/test/proper",path)
    img = load_img(img_path, grayscale=False, color_mode='rgb', target_size=(32,32), interpolation='nearest')
    img = img_to_array(img)
    img /= 255 
    test_x.append(img)
    test_y.append([1,0])
    
paths = os.listdir("image/test/broken")
paths = [x for x in paths if x.endswith(".jpg")]
for path in paths:
    img_path = os.path.join("image/test/broken",path)
    img = load_img(img_path, grayscale=False, color_mode='rgb', target_size=(32,32), interpolation='nearest')
    img = img_to_array(img)
    img /= 255.
    test_x.append(img)
    test_y.append([0,1])
    
#convert to numpy array
x = np.array(x)
y = np.array(y)
test_x = np.array(test_x)
test_y = np.array(test_y)

#split train and validation
train_x, valid_x, train_y, valid_y = train_test_split(x, y, test_size=0.2, random_state=1,stratify = y)

#check array size
print(train_x.shape,train_y.shape)
print(valid_x.shape,valid_y.shape)
print(test_x.shape,test_y.shape)

(301, 32, 32, 3) (301, 2)
(76, 32, 32, 3) (76, 2)
(20, 32, 32, 3) (20, 2)


### 1-2-1. Utilizing loops while data importing for code clarity

In [0]:
parent_dir = "image"
labels = ["proper","broken"]
datasets = ["train","test"]

x = []
y = []
test_x = []
test_y = []

for dataset in datasets:
    for label in labels:
        doi = os.path.join(parent_dir,dataset,label)
        paths = os.listdir(doi)
        paths = [x for x in paths if x.endswith(".jpg")]
        for path in paths:
            img_path = os.path.join(doi,path)
            img = load_img(img_path, grayscale=False, color_mode='rgb', target_size=(32,32), interpolation='nearest')
            img = img_to_array(img)
            img /= 255.
            
            if label is "proper":
                idx = 0
            else:
                idx = 1
            
            if dataset is "train":
                x.append(img)
                y.append(idx)
            else:
                test_x.append(img)
                test_y.append(idx)

x = np.array(x)
y = np.array(y)
test_x = np.array(test_x)
test_y = np.array(test_y)

#this function converts numbers to one hot labels. ex. 0 -> [1,0,0] 1->[0,1,0], 2->[0,0,1]
y = to_categorical(y,num_classes=2)
test_y = to_categorical(test_y,num_classes=2)

train_x, valid_x, train_y, valid_y = train_test_split(x, y, test_size=0.2, random_state=1,stratify = y)

print(train_x.shape,train_y.shape)
print(valid_x.shape,valid_y.shape)
print(test_x.shape,test_y.shape)

(301, 32, 32, 3) (301, 2)
(76, 32, 32, 3) (76, 2)
(20, 32, 32, 3) (20, 2)


## 1-2. Using ImageDatagenerator

Using the ImageDatagenerator of Keras allows users to not load everything into memory\*.
It will only store where the files are, and how it will be preprocessed in training.
ImageDatagenerator will read and yield small batches of images and corresponding labels upon iteration.
This is convineient when the input dataset is large and can not be loaded into memory.

\* It can be combined with data loaded in memory. see the later section.

### 1-2-1. When the folder structure corresponds to the class label

In [0]:
#use the newest imagedatagenerator class from keras_preprocessing
#from keras.preprocessing.image import ImageDataGenerator
from keras_preprocessing.image import ImageDataGenerator

In [0]:
#for train and validation
datagen = ImageDataGenerator(
    rescale = 1/255.,
    validation_split=0.2
)

#for test
test_datagen= ImageDataGenerator(
    rescale = 1/255.
)

print("preparing generators")

train_generator = datagen.flow_from_directory(
    directory = "image/train",
    target_size=(32,32),
    batch_size=32, #the number of images included should be divisable by the batch_size as possible. run the cell once, and change the value if not.
    classes= ["proper","broken"],
    class_mode= "categorical",
    subset="training"
)

valid_generator = datagen.flow_from_directory(
    directory = "image/train",
    target_size=(32,32),
    batch_size=25,
    classes= ["proper","broken"],
    class_mode= "categorical",
    subset="validation",
)

test_generator = test_datagen.flow_from_directory(
    directory = "image/test",
    target_size=(32,32),
    batch_size=20,
    classes= ["proper","broken"],
    class_mode= "categorical",
)

preparing generators
Found 302 images belonging to 2 classes.
Found 75 images belonging to 2 classes.
Found 20 images belonging to 2 classes.



### 1-2-2. When the folder structure do not correspond to the class label (or unknown) and instead have a csv with annotation information

In [0]:
import pandas as pd
import os

In [0]:
#we read the txt files that hold the path and labels as a dataframe
df = pd.read_csv("image/train.txt",delimiter="\t",header=None,names=["path","label"])
test_df = pd.read_csv("image/test.txt",delimiter="\t",header=None,names=["path","label"])

#preview a part of the read dataframe
df[:3]


,path,label
0,train/proper/39.jpg,0
1,train/proper/85.jpg,0
2,train/proper/70.jpg,0


In [0]:
#let's convert the path to absolute path in the dataframe
#do not run the cell twice as prefix will be repeatidly added. run the above cell to reset

#absolute path of image folder
prefix = os.path.abspath("image") + "/"

df["path"] = prefix + df["path"]
test_df["path"] = prefix + test_df["path"]

test_df[:3]

,path,label
0,/content/image/test/proper/5.jpg,0
1,/content/image/test/proper/3.jpg,0
2,/content/image/test/proper/10.jpg,0


In [0]:
#for train and validation
datagen = ImageDataGenerator(
    rescale = 1/255.,
    validation_split=0.2
)

#for test
test_datagen= ImageDataGenerator(
    rescale = 1/255.
)

print("preparing generators")

train_generator = datagen.flow_from_dataframe(
    dataframe = df,
    x_col="path",
    y_col="label",
    target_size=(32,32),
    batch_size=35, #the number of images included should be divisable by the batch_size as possible. run the cell once, and change the value if not.
    class_mode= "categorical",
    subset="training",
)

valid_generator = datagen.flow_from_dataframe(
    dataframe = df,
    x_col="path",
    y_col="label",
    target_size=(32,32),
    batch_size=25,
    class_mode= "categorical",
    subset="validation",
)
test_generator = test_datagen.flow_from_dataframe(
    dataframe = test_df,
    x_col="path",
    y_col="label",
    target_size=(32,32),
    batch_size=20,
    class_mode= "categorical"
)

preparing generators
Found 302 images belonging to 2 classes.
Found 75 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


# 2. Build the Convolutional Neural Network

## 2-1. Sequential Model way (1)

In [0]:
import keras.backend as K
from keras.models import Sequential
from keras import layers

In [5]:
K.clear_session()#not must, but preferred if you are repeatingly running model construction cells for resetting.

model = Sequential()

model.add(
    layers.Conv2D(16,(3,3),input_shape=(32,32,3),name="conv1",activation="relu",padding="same")
)
model.add(layers.MaxPool2D((2,2),name="pool1"))
model.add(layers.Dropout(0.2))
model.add(layers.Conv2D(32,(3,3),name="conv2",padding="same"))   
#activation layer can be seperately added
model.add(layers.Activation("relu"))
model.add(layers.MaxPool2D((2,2),name="pool2"))
model.add(layers.Dropout(0.2))
model.add(layers.Flatten())
model.add(layers.Dense(32,activation="relu"))
model.add(layers.Dense(2))
model.add(layers.Activation("softmax",name="preprediction"))

model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["acc"])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 32, 32, 16)        448       
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 16, 16, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 16)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 16, 16, 32)        4640      
_________________________________________________________________
activation_1 (Activation)    (None, 16, 16, 32)        0         
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 8, 8, 32)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 32)          0         
__________

## 2-2. Sequential Model way (2)

In [0]:
import keras.backend as K
from keras.models import Sequential
from keras import layers

In [0]:
K.clear_session()#not must, but preferred if you are repeatingly running model construction cells for resetting.


model = Sequential([
    layers.Conv2D(16,(3,3),name="conv1",input_shape=(32,32,3),activation="relu",padding="same"),   
    layers.MaxPool2D((2,2),name="pool1"),
    layers.Dropout(0.2),
    layers.Conv2D(32,(3,3),name="conv2",padding="same"),
    layers.Activation("relu"),
    layers.MaxPool2D((2,2),name="pool2"),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(32,activation="relu"),
    layers.Dense(2),
    layers.Activation("softmax",name="preprediction")
]
)

model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["acc"])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 32, 32, 16)        448       
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 16, 16, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 16)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 16, 16, 32)        4640      
_________________________________________________________________
activation_1 (Activation)    (None, 16, 16, 32)        0         
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 8, 8, 32)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 32)          0         
__________

## 2-3. Functional API way

In [0]:
import keras.backend as K
from keras.models import Model
from keras import layers

In [0]:
K.clear_session()#not must, but preferred if you are repeatingly running model construction cells for resetting.

inputs = layers.Input(shape=(32,32,3))
l = layers.Conv2D(16,(3,3),name="conv1",input_shape=(32,32,3),activation="relu",padding="same")(inputs)
l = layers.MaxPool2D((2,2),name="pool1")(l)
l = layers.Dropout(0.2)(l)
l = layers.Conv2D(32,(3,3),name="conv2",padding="same")(l)
l = layers.Activation("relu")(l)
l = layers.MaxPool2D((2,2),name="pool2")(l)
l = layers.Dropout(0.2)(l)
l = layers.Flatten()(l)
l = layers.Dense(32,activation="relu")(l)
l = layers.Dense(2)(l)
outputs = layers.Activation("softmax",name="preprediction")(l)

model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["acc"])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 32, 32, 16)        448       
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 16, 16, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 16)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 16, 16, 32)        4640      
_________________________________________________________________
activation_1 (Activation)    (None, 16, 16, 32)        0         
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 8, 8, 32)          0         
__________

# 3. Training 

## If dataset is loaded in memory

In [0]:
history = model.fit(train_x, train_y, batch_size=32, epochs=5, validation_data = (valid_x,valid_y))

Train on 301 samples, validate on 76 samples
Epoch 1/5
301/301 [==============================] - 1s 2ms/step - loss: 0.5311 - acc: 0.8206 - val_loss: 0.2263 - val_acc: 1.0000
Epoch 2/5
301/301 [==============================] - 0s 256us/step - loss: 0.0622 - acc: 0.9967 - val_loss: 0.0105 - val_acc: 1.0000
Epoch 3/5
301/301 [==============================] - 0s 259us/step - loss: 0.0174 - acc: 0.9900 - val_loss: 0.0210 - val_acc: 1.0000
Epoch 4/5
301/301 [==============================] - 0s 261us/step - loss: 0.0164 - acc: 0.9934 - val_loss: 0.0078 - val_acc: 1.0000
Epoch 5/5
301/301 [==============================] - 0s 255us/step - loss: 0.0113 - acc: 0.9934 - val_loss: 0.0059 - val_acc: 1.0000


## If dataset is prepared by ImageDataGenerator

In [0]:
history = model.fit_generator(
    generator = train_generator,
    steps_per_epoch = (train_generator.samples // train_generator.batch_size),
    epochs = 5,
    validation_data = valid_generator, 
    validation_steps = (valid_generator.samples // valid_generator.batch_size)
)

Epoch 1/5
9/9 [==============================] - 1s 61ms/step - loss: 0.4961 - acc: 0.8217 - val_loss: 0.2562 - val_acc: 1.0000
Epoch 2/5
9/9 [==============================] - 0s 10ms/step - loss: 0.1025 - acc: 0.9930 - val_loss: 0.0229 - val_acc: 1.0000
Epoch 3/5
9/9 [==============================] - 0s 10ms/step - loss: 0.0222 - acc: 0.9965 - val_loss: 0.0012 - val_acc: 1.0000
Epoch 4/5
9/9 [==============================] - 0s 19ms/step - loss: 0.0111 - acc: 0.9965 - val_loss: 2.1675e-04 - val_acc: 1.0000
Epoch 5/5
9/9 [==============================] - 0s 17ms/step - loss: 0.0103 - acc: 0.9930 - val_loss: 0.0023 - val_acc: 1.0000


# 4. Miscellaneous

## 4-1. Evaluate the model

### 4-1-1. If dataset is loaded in memory

In [0]:
evaluation = model.evaluate(test_x,test_y)

print(model.metrics_names) #in default, it will output loss and accuracy
print(evaluation)

20/20 [==============================] - 0s 255us/step
['loss', 'acc']
[0.00026693689869716763, 1.0]


### 4-1-2. If dataset is prepared by ImageDataGenerator

In [0]:
evaluation = model.evaluate_generator(test_generator,steps=test_generator.samples // test_generator.batch_size)

print(model.metrics_names) #in default, it will output loss and accuracy
print(evaluation)

['loss', 'acc']
[0.00026693689869716763, 1.0]


## 4-2. Use the model

### 4-2-1. Single Image

In [0]:
img = test_x[0] #the first image in the test dataset
print("shape of img is:", img.shape)

#add batch dimension so that the model can read it.

img = img[np.newaxis,...]
#or 
#img = np.expand_dims(img,axis=-1)
print("shape of modified img is:", img.shape)

predictions = model.predict(img)
predictions_label = np.argmax(predictions, axis=1)

print("shape of predictions is:", predictions.shape)
print("shape of prediction label is:", predictions_label.shape)

print("raw prediction result is:", predictions[0])
print("prediction label is:", predictions_label[0])
print("correct label is:", np.argmax(test_y[0],axis=0))

shape of img is: (32, 32, 3)
shape of modified img is: (1, 32, 32, 3)
shape of predictions is: (1, 2)
shape of prediction label is: (1,)
raw prediction result is: [9.9994290e-01 5.7056743e-05]
prediction label is: 0
correct label is: 0


### 4-2-2. Multiple Images

In [0]:
img = test_x[0:5] #the first five image in the test dataset
print("shape of img is:",img.shape)

predictions = model.predict(img)
predictions_label = np.argmax(predictions, axis=1)

print("shape of predictions is:", predictions.shape)
print("shape of prediction label is:", predictions_label.shape)
print("____")
print("raw prediction result is:", predictions)
print("____")
print("prediction label is:", predictions_label)

print("correct label is:", np.argmax(test_y[0:5],axis=1))

shape of img is: (5, 32, 32, 3)
shape of predictions is: (5, 2)
shape of prediction label is: (5,)
____
raw prediction result is: [[9.9994290e-01 5.7056692e-05]
 [9.9999857e-01 1.3985929e-06]
 [9.9999893e-01 1.0151450e-06]
 [1.0000000e+00 4.5575961e-09]
 [9.9999976e-01 2.8438191e-07]]
____
prediction label is: [0 0 0 0 0]
correct label is: [0 0 0 0 0]
